# KNN

## Variantes


- RadiusNeighborsClassifier: In cases where the data is not uniformly sampled. The user specifies a fixed radius r, such that points in sparser neighborhoods use fewer nearest neighbors for the classification. For high-dimensional parameter spaces, this method becomes less effective due to the so-called “curse of dimensionality”.
- KNeighborsClassifier: implements learning based on the k nearest neighbors of each query point, where k is an integer value specified by the user. In general, a larger k suppresses the effects of noise, but makes the classification boundaries less distinct.

# sklearn.neighbors.DistanceMetric


Metrics intended for integer-valued vector spaces: Though intended for integer-valued vectors, these are also valid metrics in the case of real-valued vectors.
identifier	class name	distance function
“hamming”	HammingDistance	N_unequal(x, y) / N_tot
No tan beneficioso: que si tenemos un dato donde aparece 10 veces la palabra 'viagra' y otro donde aparece 11 veces, no lo considera como cercano. (Por ahi tengamos que revisar los datos y separar en categorias: 'no aparece', 'aparece hasta 3 veces' y 'aparece mas de 3')
“canberra”	CanberraDistance	sum(|x - y| / (|x| + |y|))
“braycurtis”	BrayCurtisDistance	sum(|x - y|) / (sum(|x|) + sum(|y|))


En el caso que lleguemos a cambiar a los vectores por vectores booleanos: (no las mire, solo copie porque no creo que hagamos esto.)

Metrics intended for boolean-valued vector spaces: Any nonzero entry is evaluated to “True”. In the listings below, the following abbreviations are used:
N : number of dimensions
NTT : number of dims in which both values are True
NTF : number of dims in which the first value is True, second is False
NFT : number of dims in which the first value is False, second is True
NFF : number of dims in which both values are False
NNEQ : number of non-equal dimensions, NNEQ = NTF + NFT
NNZ : number of nonzero dimensions, NNZ = NTF + NFT + NTT
identifier	class name	distance function
“jaccard”	JaccardDistance	NNEQ / NNZ
“maching”	MatchingDistance	NNEQ / N
“dice”	DiceDistance	NNEQ / (NTT + NNZ)
“kulsinski”	KulsinskiDistance	(NNEQ + N - NTT) / (NNEQ + N)
“rogerstanimoto”	RogersTanimotoDistance	2 * NNEQ / (N + NNEQ)
“russellrao”	RussellRaoDistance	NNZ / N
“sokalmichener”	SokalMichenerDistance	2 * NNEQ / (N + NNEQ)
“sokalsneath”	SokalSneathDistance	NNEQ / (NNEQ + 0.5 * NTT)




Algo que se me ocurrio es que podemos hacer es mezclar un poco, separar el data frame entre los atributos booleanos y los que no lo son. Hacer KNN con ambos datos por separado y despues unir los resultados con algun peso para cada dato, pero no se. Lo vemos.

In [1]:
# Esto agrega al python path el directorio ..
import sys
sys.path.append("..")

import numpy as np
import config
from helpers import get_scores
from sklearn.grid_search import RandomizedSearchCV
from transformers import transformer
from data_builder import load_test_data, load_dev_data, load_small_dev_data

df, target = load_dev_data()
X = transformer.fit_transform(df)

print(X.shape)

(80993, 273)


In [2]:
import os

file_path = os.path.join(config.data_dir, "dev_matrix.npy")

np.save(file_path, X)

In [3]:
Y = np.load(file_path)



# Optimización de hiperparámetros

Busquemos los mejores (posibles) hiperparámetros

Para eso, primero veamos qué hiperparámetros nos provee la implementación de SKLearn.

In [4]:
from sklearn.neighbors import KNeighborsClassifier

help(KNeighborsClassifier)

Help on class KNeighborsClassifier in module sklearn.neighbors.classification:

class KNeighborsClassifier(sklearn.neighbors.base.NeighborsBase, sklearn.neighbors.base.KNeighborsMixin, sklearn.neighbors.base.SupervisedIntegerMixin, sklearn.base.ClassifierMixin)
 |  Classifier implementing the k-nearest neighbors vote.
 |  
 |  Read more in the :ref:`User Guide <classification>`.
 |  
 |  Parameters
 |  ----------
 |  n_neighbors : int, optional (default = 5)
 |      Number of neighbors to use by default for :meth:`k_neighbors` queries.
 |  
 |  weights : str or callable
 |      weight function used in prediction.  Possible values:
 |  
 |      - 'uniform' : uniform weights.  All points in each neighborhood
 |        are weighted equally.
 |      - 'distance' : weight points by the inverse of their distance.
 |        in this case, closer neighbors of a query point will have a
 |        greater influence than neighbors which are further away.
 |      - [callable] : a user-defined functi

# En clase Agustin dijo :

- Técnica simple que a veces permite aproximar conceptos muy complejos.
- El entrenamiento es muy rápido. (¿Entrenamiento?)
- La consulta es muy lenta. (AyED eficientes.)
- El modelo (¿modelo?) ocupa mucho espacio en disco.
- Para pensar: La distancia se calcula con todos los atributos. ¿Qué pasa si algunos son irrelevantes?

Lo unico que se me ocurrio para responder es que se podra ahorrar bastante tiempo si no consideramos los atributos que estan al pepe. Para el informe estaria bueno volver a tirar knn despues de haber hecho pca o ica sobre los atributos y poner un timer a ver que onda.
Uds piensan que sera eso?

Opciones:

- n_neighbors: Cantidad de vecinos (?)
- weights: str
- splitter: No estoy muy seguro
- max_depth: máxima profundidad del árbol
- max_features: qué porcentaje de variables tomo a la hora de partir un nodo.
  Estas variables se eligen aleatoriamente
- min_samples_split: cuántos elementos tengo que tener en un nodo para decidir
  partirlo

In [5]:
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV

options = {
    'n_neighbors': range(100, 1000),
    'weights': ['distance'],
    'metric' : ['canberra', 'braycurtis']
    
}


search_options = {
    'cv': 10,
    'scoring': 'roc_auc',
    'n_jobs': -1,
    'n_iter': 10,
}

search = RandomizedSearchCV(KNeighborsClassifier(), options, verbose=1, **search_options)

search.fit(X, target)

print("Mejores parámetros: {}".format(search.best_params_))

get_scores(search.best_estimator_, transformer)    

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 125.2min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 223.6min finished


Mejores parámetros: {'n_neighbors': 215, 'metric': 'canberra', 'weights': 'distance'}


,precision_score,accuracy_score,f1_score,recall_score,roc_auc_score
KNeighborsClassifier,0.970549,0.968667,0.968604,0.966667,0.968667


In [ ]:
import pickle

s = pickle.dumps(transformer)

In [ ]:
import os

print "Salvando a %s" % os.path.join(config.data_dir, "transformer.pkl")